- Load Kalman Data
- Load Video
- Align Timestamps\
- align "0" distance from ground plane on kalman\
- For each frame in video, let POS = closest kalman timestep\
- project points from "depth" where depth is render of ground plane - to test this, produce video of ground plane from phone orientation\
- rotate pointcloud and render from above
- use SIFT to transform it into relative position of first image



In [13]:
import os
import cv2
import numpy as np
import cupy as cp
from ultralytics import YOLO
import matplotlib.pyplot as plt
import sys
from PIL import Image
from importlib import reload
import pandas as pd
import plotly.express as px
from filterpy.kalman import KalmanFilter
import gdown
import zipfile

sys.path.insert(1, '..')
os.environ["OPENCV_IO_ENABLE_OPENEXR"] = "1"

In [14]:
files = {
    'walk_to_malachowski' : 'https://drive.google.com/file/d/1EejwRl80fnWIlh9l_YjYTrjQb3d7mMSR/view?usp=drive_link',
    'walk_down_parking_lot' : 'https://drive.google.com/file/d/1RtRjmR7ggn5DgyxNK6mGkihZY-g9NyCq/view?usp=drive_link',
    'malachowski_chamomile' : 'https://drive.google.com/file/d/1V28HPGOMnI-l-GiAFGB5xzt0YajLtwMU/view?usp=drive_link',
    'malachowski_chamomile_2' : 'https://drive.google.com/file/d/1JD9SI-bM44BPeG2frwaznO2Fq97BAmUM/view?usp=drive_link',
    'first_test_walk' : 'https://drive.google.com/file/d/1u7FCFc4uaZscHvZ6wH9wU2pb_Mz9djPF/view?usp=drive_link'
}

def convert_to_direct_download(gdrive_link):
    start = gdrive_link.find('/d/') + 3
    end = gdrive_link.find('/view', start)
    file_id = gdrive_link[start:end]

    direct_download_url = f'https://drive.google.com/uc?id={file_id}&export=download'

    return direct_download_url

In [15]:
# @title Selected File
selected = "walk_to_malachowski" # @param ["malachowski_chamomile", "malachowski_chamomile_2", "walk_to_malachowski", "first_test_walk", "walk_down_parking_lot"]

url = convert_to_direct_download(files[selected])
gdown.download(url, "of.zip")
with zipfile.ZipFile("./of.zip", 'r') as zip_ref:
    zip_ref.extractall("../Data/")

Downloading...
From (original): https://drive.google.com/uc?id=1EejwRl80fnWIlh9l_YjYTrjQb3d7mMSR&export=download
From (redirected): https://drive.google.com/uc?id=1EejwRl80fnWIlh9l_YjYTrjQb3d7mMSR&export=download&confirm=t&uuid=50641b51-3d48-4cb3-83db-8246105f3ab6
To: c:\Users\c1a55\Documents\Roborouter\Notebooks\of.zip
100%|██████████| 536M/536M [00:10<00:00, 52.3MB/s] 


In [18]:
%cd ../data/
%cd $selected

c:\Users\c1a55\Documents\Roborouter\data
c:\Users\c1a55\Documents\Roborouter\data\walk_to_malachowski


In [20]:
AccelerometerDF = pd.read_csv('Accelerometer.csv')
#zyx

BarometerDF = pd.read_csv('Barometer.csv')
#relativeAltitude
#pressure

#Highly prefused, but not with accel or baro
LocationDF = pd.read_csv('Location.csv')
#altitude
#speedAccuracy
#bearingAccuracy
#latitude
#altitudeAboveMeanSaLevel
#bearing
#horizontalAccuracy
#verticalAccuracy
#longitude
#speed

#Don't care, prefused into orientation
#MagnetometerDF = pd.read_csv('Magnetometer.csv')
#zyx

#To be used for synchronization
MicrophoneDF = pd.read_csv('Microphone.csv')
#dBFS

#Prefused = best guess. Smoothing welcomed.
OrientationDF = pd.read_csv('Orientation.csv')
#yaw
#qx
#qz
#roll
#qw
#qy
#pitch

In [23]:
from scipy.spatial.transform import Rotation as R
pi = np.pi

def convert_xyz_acceleration_to_ENU(ax, ay, az, qw, qx, qy, qz):
  Rquat = R.from_quat([qw, qx, qy, qz])
  rotation_matrix = Rquat.as_matrix()
  inv = rotation_matrix.T
  result = inv @ [ax, ay, az]
  return result


In [24]:
#Preprocessing the accelerations into ENU accelerations using the phone rotation:
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation as R

def convert_accelerations(accel_df, orient_df):
    # Initialize an empty DataFrame to store the converted accelerations
    converted_accelerations = pd.DataFrame(accel_df)
    converted_accelerations.drop(axis='columns', columns=['time', 'seconds_elapsed'], inplace=True)
    converted_accelerations.rename(
        mapper = {'z':'east', 'y':'north', 'x':'up'},
        axis='columns',
        inplace=True
    )

    for i in range(len(accel_df)):
        # Create a rotation object from the roll, pitch, and yaw
        qw = orient_df.iloc[i]["qw"]
        qx = orient_df.iloc[i]["qx"]
        qy = orient_df.iloc[i]["qy"]
        qz = orient_df.iloc[i]["qz"]

        # Apply the rotation to the acceleration vector
        north, east, up = convert_xyz_acceleration_to_ENU(ax, ay, az, qw, qx, qy, qz)

        # Append the converted acceleration to the DataFrame
        converted_accelerations.iloc[i] = [east, north, up]

    return converted_accelerations

ENUAccelDF = convert_accelerations(AccelerometerDF, OrientationDF)


def integrate_acceleration_to_position(accel_df, delta_t=1):
    velocity_df = pd.DataFrame(np.zeros_like(accel_df.values), columns=['east', 'north', 'up'])
    position_df = pd.DataFrame(np.zeros_like(accel_df.values), columns=['east', 'north', 'up'])

    # Integrate acceleration to get velocity and position
    for i in range(1, len(accel_df)):
        velocity_df.iloc[i] = velocity_df.iloc[i - 1] + accel_df.iloc[i] * delta_t
        position_df.iloc[i] = position_df.iloc[i - 1] + velocity_df.iloc[i] * delta_t

    return position_df

int_location_df = integrate_acceleration_to_position(ENUAccelDF, .01)
fig = px.line_3d(
    int_location_df,
    x=int_location_df['north'],
    y=int_location_df['east'],
    z=int_location_df['up']
)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed